In [5]:
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

## Models

### Vanila Large Language Models (LLMs)

LLMs are primarily designed for generating contextually relevant text, with primary focus on generating, completing, and language understanding. These models are pre-trained on diverse corpus capturing linguistic patterns for language understanding. They are widely used for downstream tasks like translation, summarization, task/domain-specific fine-tuning. etc.

Some prominent examples:
- GPT-3 (deprecated)
- llama, llama-2, llama-3

#### Loading open-source chat models using Ollama.
Using Ollama one can setup server for quantized models locally.

References:
1. [langchain](https://python.langchain.com/v0.1/docs/modules/model_io/)
2. [ollama github](https://github.com/ollama/ollama?tab=readme-ov-file)
3. [ollama model library](https://ollama.com/library)

In [ ]:
from langchain_community.llms import Ollama

llama3 = Ollama(model="llama3:text")

In [ ]:
print(llama3.invoke("What is the meaning of life in 10 words?"))

### Chat or Instruction tuned Models

Chat or instruction models are specifically designed for following user instructions or engaging in conversation with the user. They are LLMs that are further fine-tuned with specific datasets. Their main focus is to understand the context from user queries and respond accordingly. They are widely used for question answering, chatbots, dialogoe systems, etc.

Some prominent examples:
- GPT-3.5-turbo, GPT-4
- llama-chat models
- claude-2

In langchain, a chat model is a language model that uses chat messages as inputs and returns chat messages as outputs.

##### Passing user message to model through HumanMessage

In [ ]:
from langchain_core.messages import HumanMessage
message = [HumanMessage("What is the meaning of life in 10 words?")]

#### OpenAI models

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
chat_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,
    max_tokens=1024
)

`invoke()` call the chain on an input

In [ ]:
print(chat_llm.invoke(message))

`stream()` stream back chunks of the response

In [ ]:
for chunk in chat_llm.stream(message):
    print(chunk.content, end="", flush=True)

In [ ]:
llm_name= "gpt-4"
chat_llm_gpt4 = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=openai.api_key)

In [ ]:
print(chat_llm_gpt4.invoke(message))

P.S.: The LLM returns a string, while the ChatModel returns a message.

#### Loading open-source chat models using Ollama.

In [ ]:
from langchain_community.chat_models import ChatOllama

llama3_chat = ChatOllama(model="llama3",
                         temperature=0.0,
                         max_tokens=1024,
                         top_k=10,)

In [ ]:
print(llama3_chat.invoke(message).content)

## Prompts and Prompt Templates

A **prompt** could be an instruction or a query that is passed to the llm. At times, it can also contain some more details in the form of context, input, or example.

A **prompt template** is a wrapper around user-prompt providing extra layer of information specific to model and task. With prompt template user input can become more dynamic, as it can provide a placeholder.

### PromptTemplate

`PromptTemplate` is used to create a template for a string prompt.

Important Functions:
- `PromptTemplate.from_template()` to load a prompt template from a template.
- `PromptTemplate.format()` to format the defined template with user input. ==> Format the chat template into a string.

Reference: [langchain PromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/#prompttemplate)

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
prompt = PromptTemplate.from_template("What is the meaning of life in less than {num_of_words} words {style}?")
print(prompt.format(num_of_words=100, style=""))

In [ ]:
prompt

In [ ]:
print(llama3.invoke(prompt.format(num_of_words=10, style="")))

In [ ]:
print(llama3.invoke(prompt.format(num_of_words=50, style="")))

In [ ]:
print(llama3.invoke(prompt.format(num_of_words=50, style="in a royal way")))

### ChatPromptTemplate

`ChatPromptTemplate`, prompt template for chat models, is a list of `ChatMessageTemplates`. Each `ChatMessageTemplate` contains instructions for how to format that `ChatMessage` - its role, and then also its content.

Important Classes:
- `SystemMessagePromptTemplate`
- `SystemMessage`: This represents a system message, which tells the model how to behave. This generally only consists of content. Not every model supports this.
- `HumanMessagePromptTemplate`
- `HumanMessage`: This represents a message from the user. Generally consists only of content.

Important Functions:
- `ChatPromptTemplate.from_messages()` defines the chat template. Most commonly used with `ChatPromptTemplate`. ==> Create a chat prompt template from a variety of message formats.
- `ChatPromptTemplate.format_messages()` to format the defined template with user input. ==> Format the chat template into a list of finalized messages.

Reference: 
- [langchain ChatPromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/#chatprompttemplate)
- [OpenAI ChatCOmpletion](https://platform.openai.com/docs/guides/text-generation/chat-completions-api)

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is the meaning of life in less than {num_of_words} words {style}?")
message = prompt.format(num_of_words=50, style="in a funny way")

In [ ]:
print(message)
print(type(message))

default message becomes `HumanMessage`. This represent user instruction.

In [ ]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_message = chat_prompt.format_messages(input_language="English", 
                            output_language="Hindi", 
                            text="The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.")

In [ ]:
print(chat_message)
print(type(chat_message))
for msg in chat_message:
    print(msg, type(msg))

In [ ]:
print(chat_llm.invoke(chat_prompt.format_prompt(input_language="English", 
                            output_language="Hindi", 
                            text="The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.")))

#### Using Placeholder

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

human_template = "Summarise the converstion in {word_count} words."
humman_message_template = HumanMessagePromptTemplate.from_template(human_template)
print(humman_message_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"), humman_message_template]
)
print(chat_prompt)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
system_message = SystemMessage(content="You are a smart AI assistant.")
human_message = HumanMessage(content="What is the meaning of life in less than 50 words?")
ai_message = AIMessage(
    content="""The meaning of life is to find joy and purpose in living, and to make a positive impact on the world."""
)

chat_message = chat_prompt.format_messages(
    conversation=[system_message, human_message, ai_message], word_count=20,
)
print(chat_message)

In [ ]:
print(chat_llm_gpt4.invoke(chat_message))

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt=chat_prompt,
    llm=chat_llm,
    verbose=True)
chain.predict(conversation=[system_message, human_message, ai_message], word_count=20)

For more examples, reference [langchain docs](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html), [langchain tutorials](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/)